# Step 1: Data Preprocessing Pipeline

### Importing Libraries

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
import joblib

In [ ]:
!pip install fastapi

In [ ]:
from fastapi import FastAPI

In [ ]:
from pydantic import BaseModel

## From fastapi import FastAPI
### Step 1: Identify Features

In [ ]:
target = 'Sleep_Duration'

In [ ]:
categorical = ['Gender']

In [ ]:
numerical = ['Age', 'Caffeine_Intake', 'Screen_Time', 'Physical_Activity_Level']

### Custom Transformer for Interaction Features

In [ ]:
class InteractionFeatures(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        if 'Screen_Time' in X and 'Caffeine_Intake' in X:
            X['Screen_Caffeine_Interaction'] = X['Screen_Time'] * X['Caffeine_Intake']
        return X

### Preprocessing Pipeline

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='first'))
])

### Full Column Transformer

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numerical),
    ('cat', categorical_transformer, categorical)
])

### Polynomial Features

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)

### Full Pipeline with Feature Engineering

In [ ]:
full_pipeline = Pipeline(steps=[
    ('interaction', InteractionFeatures()),
    ('preprocess', preprocessor),
    ('poly', poly)
])

### Define the DataFrame with necessary columns

In [ ]:
df = pd.read_csv('data.csv')

### Split the data into features and target

In [ ]:
X = df.drop(columns=[target])

In [ ]:
y = df[target]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Model Development